In [1]:
import numpy as np
import pandas as pd

from copy import deepcopy
from functools import partial
import matplotlib.pyplot as plt
import optuna
import pickle
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

In [2]:
# leaked_df = pd.read_csv('../input/leaked_data_all.csv', parse_dates=['timestamp'])
with open('../input/leak_data_drop_bad_rows.pkl', 'rb') as f:
    leaked_df = pickle.load(f).rename(columns={'meter_reading': 'leaked_meter_reading'})

# leaked_df = pd.read_feather('../input/leak_data.feather').rename(columns={'meter_reading': 'leaked_meter_reading'})
leaked_df = leaked_df[['building_id','meter','timestamp', 'leaked_meter_reading']]
leaked_df = leaked_df.query('timestamp>=20170101')

building_meta = pd.read_csv("../input/building_metadata.csv")

leaked_df = leaked_df.merge(building_meta[['building_id', 'site_id']], on='building_id', how='left')

leaked_df = leaked_df.query('~(meter==0 & site_id==0)')
# leaked_df = leaked_df.query('site_id==[2,4,15]')
# leaked_df = leaked_df.query('105<=building_id<=564 | 656<=building_id')

test = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])

In [3]:
i = 1

for mul in tqdm(['05', '10', '15']):
    submission_s1 = pd.read_csv(f'../output/use_train_fe_seed1_leave31_lr005_tree500_mul{mul}.csv')
    submission_s2 = pd.read_csv(f'../output/use_train_fe_seed2_leave31_lr005_tree500_mul{mul}.csv')
    submission_s3 = pd.read_csv(f'../output/use_train_fe_seed3_leave31_lr005_tree500_mul{mul}.csv')
    test[f'pred{i}'] = (submission_s1['meter_reading'] + submission_s2['meter_reading'] + submission_s3['meter_reading']) / 3
#             test[f'pred{i}'] = submission_s1['meter_reading']
    i += 1
# del submission_s1, submission_s2, submission_s3

100%|██████████| 3/3 [01:40<00:00, 33.42s/it]


In [4]:
for tree in [500]:
    for leave in tqdm([7]):
        for lr in ["005"]:
            submission_s1 = pd.read_csv(f'../output/use_train_fe_seed1_leave{leave}_lr{lr}_tree{tree}_mul05.csv')
            submission_s2 = pd.read_csv(f'../output/use_train_fe_seed2_leave{leave}_lr{lr}_tree{tree}_mul05.csv')
            submission_s3 = pd.read_csv(f'../output/use_train_fe_seed3_leave{leave}_lr{lr}_tree{tree}_mul05.csv')
            test[f'pred{i}'] = (submission_s1['meter_reading'] + submission_s2['meter_reading'] + submission_s3['meter_reading']) / 3
#             test[f'pred{i}'] = submission_s1['meter_reading']
            i += 1
# del submission_s1, submission_s2, submission_s3

100%|██████████| 1/1 [00:32<00:00, 32.91s/it]


In [5]:
# for tree in [500]:
#     for leave in tqdm([7, 15]):
#         for lr in ["001", "005","01"]:
#             submission_s1 = pd.read_csv(f'../output/use_train_fe_noTE_seed1_leave{leave}_lr{lr}_tree{tree}_mul05.csv')
# #             submission_s2 = pd.read_csv(f'../output/use_train_fe_noTE_seed2_leave{leave}_lr{lr}_tree{tree}_mul05.csv')
# #             submission_s3 = pd.read_csv(f'../output/use_train_fe_noTE_seed3_leave{leave}_lr{lr}_tree{tree}_mul05.csv')
# #             test[f'pred{i}'] = (submission_s1['meter_reading'] + submission_s2['meter_reading'] + submission_s3['meter_reading']) / 3
#             test[f'pred{i}'] = submission_s1['meter_reading']
#             i += 1
# # del submission_s1, submission_s2, submission_s3

In [6]:
# # for name in ['fe2_lgbm', 'submission_tomioka', 'submission_half_and_half', 'submission_distill', 'submission_TE_50000tree_seed1_mul075']:
# for name in ['half_and_half', 'submission_simple_data_cleanup']:#, 'use_train_fe_seed1_leave15_lr001_tree20000_mul05']:#, 'fe2_lgbm']:
#     print(i, end=' ')
#     test[f'pred{i}'] = pd.read_csv(f'../output/{name}.csv')['meter_reading']
#     i += 1

5 

FileNotFoundError: [Errno 2] File b'../old_submission/half_and_half.csv' does not exist: b'../old_submission/half_and_half.csv'

In [8]:
# for name in ['fe2_lgbm', 'submission_tomioka', 'submission_half_and_half', 'submission_distill', 'submission_TE_50000tree_seed1_mul075']:
for name in ['submission_half_and_half', 'submission_simple_data_cleanup']:#, 'use_train_fe_seed1_leave15_lr001_tree20000_mul05']:#, 'fe2_lgbm']:
    print(i, end=' ')
    test[f'pred{i}'] = pd.read_csv(f'../old_submission/{name}.csv')['meter_reading']
    i += 1

5 6 

In [9]:
test[f'pred{i}'] = np.exp(1) - 1
i += 1

In [10]:
test = test.merge(leaked_df, on=['building_id', 'meter', 'timestamp'], how='left')
N = test.columns.str.startswith('pred').sum()
print(N)

7


In [11]:
test_sub = test.copy()
test = test[~test['leaked_meter_reading'].isnull()]
test2017 = test.query('timestamp<20180101')
test2018 = test.query('20180101<=timestamp')

In [12]:
for i in range(1,N+1):
    print(i, np.sqrt(mean_squared_error(np.log1p(test2018['leaked_meter_reading']), np.log1p(test2018[f'pred{i}']))))

print()

for i in range(1,N+1):
    print(i, np.sqrt(mean_squared_error(np.log1p(test2017['leaked_meter_reading']), np.log1p(test2017[f'pred{i}']))))

print()
    
for i in range(1,N+1):
    print(i, np.sqrt(mean_squared_error(np.log1p(test['leaked_meter_reading']), np.log1p(test[f'pred{i}']))))


1 1.0545973830481492
2 1.0554909788652922
3 1.056746048041055
4 1.0581681178741322
5 1.083063385436808
6 1.0839501550420507
7 3.695562019317608

1 0.9138296903345354
2 0.9097899973322123
3 0.9097473660053049
4 0.9245811185333753
5 0.9460219466177832
6 0.9613417629834499
7 3.7055726492739844

1 0.986605695739567
2 0.9852117321248618
3 0.985863335918557
4 0.9935070157080577
5 1.0167358027916829
6 1.024376067314449
7 3.7005793672849165


In [13]:
# sid = 1
# for i in range(1,N+1):
#     print(i, np.sqrt(mean_squared_error(np.log1p(test2018.query(f'meter==0 & site_id=={sid}')['leaked_meter_reading']), 
#                                         np.log1p(test2018.query(f'meter==0 & site_id=={sid}')[f'pred{i}']))))

# print()

# for i in range(1,N+1):
#     print(i, np.sqrt(mean_squared_error(np.log1p(test2017.query(f'meter==0 & site_id=={sid}')['leaked_meter_reading']), 
#                                         np.log1p(test2017.query(f'meter==0 & site_id=={sid}')[f'pred{i}']))))
       

In [14]:
def preproceeding(submission, N):
    submission.loc[:,'pred1':'leaked_meter_reading'] = np.log1p(submission.loc[:,'pred1':'leaked_meter_reading'])
    g = submission.groupby('meter')
    sub_sub = [dict(), dict(), dict(), dict()]
    leak_sub = [dict(), dict(), dict(), dict()]
    leak_leak = [0,0,0,0]
    
    for meter in [3,2,1,0]:
        for i in tqdm(range(1,N+1)):
            leak_sub[meter][i] = sum(-2 * g.get_group(meter)['leaked_meter_reading'] * g.get_group(meter)[f'pred{i}'])
            for j in range(1,N+1):
                if i > j: 
                    sub_sub[meter][(i,j)] = sub_sub[meter][(j,i)]
                else:
                    sub_sub[meter][(i,j)] = sum(g.get_group(meter)[f'pred{i}'] * g.get_group(meter)[f'pred{j}'])
        
        leak_leak[meter] = (sum(g.get_group(meter)['leaked_meter_reading'] ** 2))
    
    return sub_sub, leak_sub, leak_leak

In [15]:
def optimization(meter, sub_sub, leak_sub, leak_leak, length, W):
    
#     global count_itr
#     if count_itr%1000 == 0: print(count_itr, end=' ')
#     count_itr += 1
    
    loss_total = 0

    for i, a in enumerate(W, 1):
        for j, b in enumerate(W, 1):
            loss_total += a * b * sub_sub[meter][(i, j)]

    for i, a in enumerate(W, 1):
        loss_total += leak_sub[meter][i] * a

    loss_total += leak_leak[meter]
    
    return np.sqrt(loss_total / length)

In [16]:
def make_ensemble_weight(focus_df, N):

    sub_sub, leak_sub, leak_leak = preproceeding(focus_df.copy(), N)


    np.random.seed(1)
    score = [list(), list(), list(), list()]
    weight = [list(), list(), list(), list()]

    for meter in [0,1,2,3]:
        f = partial(optimization, meter, sub_sub, leak_sub, leak_leak, len(focus_df.query(f'meter=={meter}')))
        for i in tqdm(range(1000000)):
            W = np.random.rand(N)

            to_zero = np.arange(N)
            np.random.shuffle(to_zero)

            W[to_zero[:np.random.randint(N)]] = 0
            W /= W.sum()
            W *= np.random.rand() * 0.3 + 0.8
            score[meter].append(f(W))
            weight[meter].append(W)

        score[meter] = np.array(score[meter])
        weight[meter] = np.array(weight[meter])
    
    return weight, score

In [17]:
weight2017, score2017 = make_ensemble_weight(test2017, N)

100%|██████████| 1000000/1000000 [01:09<00:00, 14482.16it/s]


In [18]:
weight2018, score2018 = make_ensemble_weight(test2018, N)

100%|██████████| 1000000/1000000 [01:12<00:00, 13877.79it/s]


In [19]:
for meter in [0,1,2,3]:
#     for i in range(N):
    print(weight2017[meter][score2017[meter].argmin()])
    print()

# for meter in [0,1,2,3]:
#     print(score2017[meter].min())
#     print(weight2017[meter][score2017[meter].argmin()].sum())
#     print()

[0.12026147 0.         0.5505669  0.         0.323073   0.
 0.        ]

[0.         0.         0.55266118 0.         0.         0.39306029
 0.09019167]

[0.         0.         0.84964604 0.         0.14769226 0.
 0.        ]

[0.18689671 0.41500583 0.         0.         0.3644072  0.
 0.        ]



In [20]:
for meter in [0,1,2,3]:
#     for i in range(N):
    print(weight2018[meter][score2018[meter].argmin()])
    print()

# for meter in [0,1,2,3]:
#     print(score2018[meter].min())
#     print(weight2018[meter][score2018[meter].argmin()].sum())
#     print()

[0.41718076 0.         0.2852213  0.         0.27707631 0.
 0.        ]

[0.         0.         0.34437608 0.13223363 0.         0.43411985
 0.1821063 ]

[0.         0.         0.31167629 0.13261928 0.2036462  0.3573865
 0.        ]

[0.         0.48347138 0.         0.         0.30641592 0.14341469
 0.        ]



In [21]:
def new_pred(test, weight, score, N):
    pred_new = list()
    for meter in [0,1,2,3]:
        test_m = test.query(f'meter=={meter}')
        ensemble_m = sum([np.log1p(test_m[f'pred{i+1}']) * weight[meter][score[meter].argmin()][i] for i in range(N)])
        pred_new.append(ensemble_m)

    pred_new = pd.concat(pred_new).sort_index()
    return np.expm1(pred_new)

In [22]:
ensembled_pred2017 = new_pred(test_sub.query('timestamp<20180101'), weight2017, score2017, N)
ensembled_pred2018 = new_pred(test_sub.query('20180101<=timestamp'), weight2018, score2018, N)

In [23]:
ensembled_pred = pd.concat([ensembled_pred2017, ensembled_pred2018], axis=0).sort_index()

In [24]:
print(np.sqrt(mean_squared_error(np.log1p(test2018['leaked_meter_reading']), np.log1p(ensembled_pred.loc[test2018.index]))))
print(np.sqrt(mean_squared_error(np.log1p(test2017['leaked_meter_reading']), np.log1p(ensembled_pred.loc[test2017.index]))))
print(np.sqrt(mean_squared_error(np.log1p(test['leaked_meter_reading']), np.log1p(ensembled_pred.loc[test.index]))))

1.0295527690956277
0.8923852193404604
0.9632950742127016


In [25]:
new_submission = pd.read_csv('../input/sample_submission.csv')

In [26]:
new_submission['meter_reading'] = ensembled_pred.values
#                                       submission1['meter_reading'] * arr[arr[:,0].argmin()][1][0] \
#                                     + submission2['meter_reading'] * arr[arr[:,0].argmin()][1][1] \
#                                     + submission3['meter_reading'] * arr[arr[:,0].argmin()][1][2]

In [27]:
new_submission.to_csv('../output/submission_my_leak_validation.csv', index=False)

In [27]:
new_submission

,row_id,meter_reading
0,0,137.404518
1,1,64.776771
2,2,5.452883
3,3,231.476378
4,4,1073.066650
5,5,8.973845
6,6,99.896591
7,7,291.478305
8,8,625.875907
9,9,307.480913
